[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# JSONSchema en Python.

https://json-schema.org/

In [ ]:
!pip install jsonschema 

In [ ]:
import jsonschema

In [ ]:
esquema_alumno = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "Nombre": {"type": "string",
                   "minLength":1,
                   },
        "Primer Apellido": {"type": "string", 
                   "minLength":1,},
        "Segundo Apellido": {"type": "string", 
                   "minLength":1,},
        "Carrera": {"type": "string"},
        "Semestre": {"type": "number",
                   "minimum": 1,
                   "maximum": 50,},
        "Promedio": {"type": "number",
                   "minimum": 0,
                   "maximum": 10,},
        "Al Corriente": {"type": "boolean"},
    },
    "required": ["Nombre",  "Primer Apellido", "Carrera", "Semestre",
                 "Promedio", "Al Corriente"]
}

In [ ]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,}, 
                    esquema_alumno)

In [ ]:
jsonschema.validate({'Al Corriente': False,
                     'Carrera': 'Arquitectura',
                     'Nombre': 'Pedro', 
                     'Primer Apellido': 'Solis', 
                     'Promedio': 7.8, 
                     'Semestre': 5,
                     'Género': 'F',},
                    esquema_alumno)

In [ ]:
jsonschema.validate({'Al Corriente': False, 
          'Carrera': 'Arquitectura', 
          'Nombre': 'Pedro', 
          'Primer Apellido': 'Solis', 
          'Promedio': 7.8, 
          'Semestre': -7,}, 
                    esquema_alumno)

## Importación de módulos y datos.

In [ ]:
from flask import Flask, jsonify, request, abort
from json import loads, dumps
import jsonschema
from data import carreras, ruta, esquema_alumno 

In [ ]:
carreras

In [ ]:
ruta

In [ ]:
esquema_alumno

## Definición de funciones.

### Funciones de gestión de la base de datos.

En este caso la base de datos no es otra cosa más que un archivo de texto que representa a un objeto de tipo *list* de Python. 

La base de datos puede ser consultada en [data/alumnos.txt](data/alumnos.txt). 

In [ ]:
def accede_base(ruta, cuenta):
    alumno = None
    with open(ruta, 'tr') as f:
        base = eval(f.read())
    for item in base:
        try:
            if item['Cuenta'] == int(cuenta):
                alumno = item
        except:
            break
    return base, alumno

## Funciones de validación de datos.

### Función que valida las reglas de los datos.

* Validará que los datos tengan una estructura que con la descripción de ```esquema_alumno```.
* E texto del campo ```'Carrera'``` debe de estar dentro de las cadenas listadas en ```data.carreras```.

In [ ]:
esquema_alumno

### Función que valida que el mensaje contiene todos los campos obligatorios.

In [ ]:
def recurso_completo(cuenta, peticion):
    try:
        peticion = loads(peticion)
        jsonschema.validate(peticion, esquema_alumno)
        if set(esquema_alumno['properties']).issuperset(peticion) \
        and peticion['Carrera'] in carreras:
            peticion['Cuenta'] = int(cuenta)
        else:
            raise(TypeError, "Not valid")
    except (jsonschema.exceptions.ValidationError, TypeError):
        return None
    return peticion

## Código del servidor.

* El servidor correrá en [localhost:5000/api/](localhost:5000/api/). Si se accede a la raíz, se desplegará un listado de todos los alumnos en formato JSON.
* El servidor soporta los métodos:
    * **GET**: para obtener la información de un alumno por su número de cuenta.
    * **POST**: para crear un registro nuevo.
    * **PUT**: para sustituir por completo un registro existente.
    * **PATCH**: para modificar ciertos datos de un registro existente.
    * **DELETE**: para eliminar un registro existente.

In [ ]:
app = Flask(__name__)


@app.route('/api/', methods=['GET'])
def index():
    with open(ruta, 'tr') as base:    
        return jsonify(eval(base.read()))

    
@app.route('/api/<cuenta>', methods=['GET', 'POST', 'PUT', 'PATCH', 'DELETE'])
def api(cuenta):
    
    if request.method == 'GET':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            return jsonify(alumno)
        else:
            abort(404)
            
    if request.method == 'DELETE':               
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            base.remove(alumno)
            with open(ruta, 'wt') as f:
                f.write(str(base))
            return jsonify(alumno)
        else:
            abort(404)
        
    if request.method == 'POST':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            abort(409)
        else:
            respuesta = recurso_completo(cuenta, request.data)
            if respuesta:
                base.append(respuesta)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(respuesta)
            else:
                abort(406)
            
    if request.method == 'PUT':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            respuesta = recurso_completo(cuenta, request.data)
            if respuesta:
                base.remove(alumno)
                base.append(respuesta)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(respuesta)
            else:
                abort(406)
        else:
            abort(404)
            
    if request.method == 'PATCH':
        base, alumno = accede_base(ruta, cuenta)
        if alumno:
            del alumno["Cuenta"]
            alumno.update(loads(request.data))
            respuesta = recurso_completo(cuenta, dumps(alumno))
            if respuesta:
                base.remove(alumno)
                base.append(respuesta)
                with open(ruta, 'wt') as f:
                    f.write(str(base))
                return jsonify(respuesta)
            else:
                abort(406)
        else:
            abort(404)

In [ ]:
app.run('0.0.0.0')

### Notas:  
* **No reinicie o detenga el kernel de la notebook hasta que los clientes que accedan a esta aplicación hayan terminado sus sesiones.**
* Debido a que el código de la celda de arriba levanta el servidor de Flask, ésta se ejecutará indefinidamente y desplegará los mensajes de respuesta a las peticiones de los clientes que se conecten. 
* La notebook [14_cliente_api_rest.ipynb](12_cliente_api_rest.ipynb) contiene al cliente para este servidor.


<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>